In [2]:
from feast import Client

# Connect to an existing Feast Core deployment
client = Client(core_url="172.20.66.56:6565", serving_url='172.20.39.61:6566', spark_launcher="k8s", spark_k8s_namespace="default")

# Ensure that your client is connected by printing out some feature tables
# project to track feature table
client.set_project("dev_test")
client.list_feature_tables()

[FeatureTable <dev_ft>]

In [3]:
# create entity
from feast import Entity, ValueType

id = Entity(name="id", description="ID of cluster", value_type=ValueType.INT64)
client.apply(id)


In [12]:
import pandas as pd
from feast import FeatureTable, Feature,ValueType, FileSource
from feast.data_format import ParquetFormat

# read input dataframe
df = pd.read_csv('/home/jovyan/feast_ingestion/userinput.csv')



# create and apply feature table
ft = FeatureTable(name='dev_ft', entities=["id"],
    features=[
      Feature("feature1", ValueType.INT32),
        Feature("feature2", ValueType.INT32),
    Feature("feature3", ValueType.INT32),
    Feature("feature4", ValueType.INT32)],
        batch_source=FileSource(
    file_format=ParquetFormat(),
    file_url="file:///home/jovyan/feast_ingestion/",
    event_timestamp_column="event_timestamp",
    created_timestamp_column="created_timestamp",
),
    )

client.apply(ft)

client.list_feature_tables()

[FeatureTable <dev_ft>]

In [13]:
import numpy as np
from datetime import datetime
df['event_timestamp'] = pd.to_datetime(
            np.random.randint(
                datetime(2020, 10, 10).timestamp(),
                datetime(2020, 10, 20).timestamp(),
                size=1),
        unit="s"
    )
df['created_timestamp'] = pd.to_datetime(datetime.now())
df

,id,ideal_min_CPU,ideal_max_CPU,ideal_min_memory,ideal_max_memory,event_timestamp,created_timestamp
0,1,1,2,1,2,2020-10-14 09:43:07,2021-03-20 10:38:08.759275


In [14]:
# ingest data into feature table 
client.ingest(ft, df)

Removing temporary file(s)...
Data has been successfully ingested into FeatureTable batch source.


In [8]:
import os
os.environ["FEAST_HISTORICAL_FEATURE_OUTPUT_LOCATION"] = "file:///home/jovyan/historical_feature_output"
os.environ["FEAST_SPARK_STAGING_LOCATION"] = "file:///home/jovyan/test_data"

In [15]:
# test receiving features

import time

# Feature references with target feature
feature_refs = [
    "dev_ft:ideal_min_cpu",
]

In [16]:
# specify entities


entities_with_timestamp = pd.DataFrame(columns=['id', 'event_timestamp'])

entities_with_timestamp['id'] = np.array([1])
entities_with_timestamp['event_timestamp'] = pd.to_datetime(datetime.now())
entities_with_timestamp

,id,event_timestamp
0,1,2021-03-20 10:38:27.993960


In [1]:
# Retrieve historical dataset from Feast.

historical_feature_retrieval_job = client.get_historical_features(
    feature_refs=feature_refs,
    entity_source=entities_with_timestamp,
)

output_file_uri = historical_feature_retrieval_job.get_output_file_uri()
output_file_uri

NameError: name 'client' is not defined